In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_car.sapcar_full_inventory
(
  hash_id BIGINT,
  calday DATE,
  calhour BIGINT,
  store_id STRING,
  product_id STRING,
  baseuom STRING,
  unreuseqty DOUBLE,
  unreconsqty DOUBLE,
  unprsaleqty DOUBLE,
  closing_stock_qty DOUBLE
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW temp_sapcar_full_inventory AS
    SELECT DISTINCT
      FARM_FINGERPRINT(CONCAT(
              IFNULL(CAST(timestamp AS STRING), ""), 
              IFNULL(CAST(site AS STRING), ""), 
              IFNULL(CAST(articlenumber AS STRING), ""), 
              IFNULL(CAST(baseuom AS STRING), "")
      )) hash_id,
      TO_DATE(TO_TIMESTAMP(CAST(timestamp AS STRING), 'yyyyMMddHHmmss')) AS calday,
      HOUR(TO_TIMESTAMP(CAST(timestamp AS STRING), 'yyyyMMddHHmmss')) AS calhour,
      CAST(site AS STRING) AS store_id,
      CAST(articlenumber AS STRING) AS product_id,
      baseuom,
      IFNULL(CAST(REPLACE(unreuseqty, ',', '.') AS DOUBLE), 0) AS unreuseqty,
      IFNULL(CAST(REPLACE(unreconsqty, ",", ".") AS DOUBLE), 0) unreconsqty,
      IFNULL(CAST(REPLACE(CAST(unprsaleqty AS STRING), ',', '.') AS DOUBLE), 0) AS unprsaleqty,
      IFNULL(CAST(REPLACE(unreuseqty, ',', '.') AS DOUBLE), 0) + 
      IFNULL(CAST(REPLACE(unreconsqty, ',', '.') AS DOUBLE), 0) - 
      IFNULL(CAST(REPLACE(CAST(unprsaleqty AS STRING), ',', '.') AS DOUBLE), 0) AS closing_stock_qty
    FROM {catalog_name}.udp_wcm_bronze_sap_car.full_sapcar;
""")

In [0]:
spark.sql("""
MERGE INTO {catalog_name}.udp_wcm_silver_sap_car.sapcar_full_inventory main_tbl
USING temp_sapcar_full_inventory temp_tbl
ON main_tbl.hash_id = temp_tbl.hash_id
WHEN NOT MATCHED THEN INSERT *
""")